In [ ]:
from node2vec import Node2Vec
import numpy as np
import networkx as nx
import networkx.algorithms.community as nx_com
import networkx.algorithms.centrality as nx_cen
import random 
import copy
import math
import csv
import matplotlib.pyplot as plt
from random import uniform, seed
import pandas as pd
import time
from collections import Counter
from igraph import Graph  
from tqdm import tqdm
from itertools import product
import itertools 
import heapq
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import genetic_algorithm
import genetic_im_opt
import celfpp
import simulated_annealing
import pickle

In [ ]:
Email_df = pd.read_csv('/content/email-Eu-core.txt.gz', compression='gzip', names=["source", "target"], sep=' ', quotechar='"', error_bad_lines=False)
fb_df = pd.read_csv('/content/facebook_combined.txt.gz', compression='gzip', names=["source", "target"], sep=' ', quotechar='"', error_bad_lines=False)
# twitter_df = pd.read_csv('/content/twitter_combined.txt.gz', compression='gzip', names=["source", "target"], sep=' ', quotechar='"', error_bad_lines=False)
# txroad_df = pd.read_csv('/content/roadNet-TX.txt',sep='\t',skiprows=(0,1,2,3),names=["source", "target"],quotechar='"',error_bad_lines=False)
# wiki_df = pd.read_csv('/content/Wiki-Vote.txt', names=["source", "target"], sep='/t', quotechar='"', error_bad_lines=False)
# Email Data to Graph
Email_graph=Graph(directed=True)
Email_graph.add_vertices(1005)
Email_graph.add_edges(zip(Email_df["source"], Email_df["target"]))
print("Number of vertices in the graph:", Email_graph.vcount())
print("Number of edges in the graph",Email_graph.ecount())
print("Is the graph directed:", Email_graph.is_directed())
print("Maximum degree in the graph:", Email_graph.maxdegree())
print('Graph Summary: ', Email_graph.summary())

# Facebook Data to Graph
fb_graph=Graph(directed=True)
fb_graph.add_vertices(4039)
fb_graph.add_edges(zip(fb_df["source"], fb_df["target"]))
print("Number of vertices in the graph:", fb_graph.vcount())
print("Number of edges in the graph",fb_graph.ecount())
print("Is the graph directed:", fb_graph.is_directed())
print("Maximum degree in the graph:", fb_graph.maxdegree())
print('Graph Summary: ', fb_graph.summary())

# # Twitter Data
# twitter_graph=Graph(directed=True)
# twitter_graph.add_vertices(81306)
# # twitter_graph.add_edges(zip(twitter_df["source"], twitter_df["target"]))
# print("Number of vertices in the graph:", twitter_graph.vcount())
# print("Number of edges in the graph",twitter_graph.ecount())
# print("Is the graph directed:", twitter_graph.is_directed())
# print("Maximum degree in the graph:", twitter_graph.maxdegree())
# print('Graph Summary: ', twitter_graph.summary())

# # Text Road Network Data
# txroad_graph=Graph(directed=True)
# txroad_graph.add_vertices(1393361)
# txroad_graph.add_edges(zip(txroad_df["source"], txroad_df["target"]))
# print("Number of vertices in the graph:", txroad_graph.vcount())
# print("Number of edges in the graph",txroad_graph.ecount())
# print("Is the graph directed:", txroad_graph.is_directed())
# print("Maximum degree in the graph:", txroad_graph.maxdegree())
# print('Graph Summary: ', txroad_graph.summary())

Emailx_graph = nx.from_pandas_edgelist(Email_df,'source','target')
fbx_graph = nx.from_pandas_edgelist(fb_df,'source','target')
nx.draw(Emailx_graph,pos=nx.spring_layout(Emailx_graph),with_labels=True)
plt.show()
nx.draw(fbx_graph,pos=nx.spring_layout(fbx_graph),with_labels=True)
plt.show()
Email_node_degree = Emailx_graph.degree()
fb_node_degree = fbx_graph.degree()
Email_n2v = Node2Vec(Emailx_graph, dimensions=2,walk_length=20,num_walks=10,seed=10,temp_folder='/content', workers=1)
fb_n2v = Node2Vec(fbx_graph, dimensions=2,walk_length=40,num_walks=20,seed=10,temp_folder='/content', workers=1)
# twitter_n2v = Node2Vec(twitterx_graph, dimensions=2,walk_length=60,num_walks=30,seed=10,temp_folder='/content', workers=1)
# txroad_n2v = Node2Vec(txroadx_graph, dimensions=2,walk_length=80,num_walks=40,seed=10,temp_folder='/content', workers=1)
Email_model = Email_n2v.fit(window=4,min_count=1,batch_words=2)
fb_model = fb_n2v.fit(window=4,min_count=1,batch_words=2)
# twitter_model = twitter_n2v.fit(window=4,min_count=1,batch_words=2)
# txroad_model = txroad_n2v.fit(window=4,min_count=1,batch_words=2)
Email_model.wv.save_word2vec_format('./Email_emb.csv')
fb_model.wv.save_word2vec_format('./fb_emb.csv')
# twitter_model.wv.save_word2vec_format('./twitter_emb.csv')
# txroad_model.wv.save_word2vec_format('./txroad_emb.csv')
Email_emb_df = pd.read_csv('/content/Email_emb.csv',sep=' ',skiprows=[0],header=None)
Email_emb_df.columns = ['source','em1','em2']

fb_emb_df = pd.read_csv('/content/fb_emb.csv',sep=' ',skiprows=[0],header=None)
fb_emb_df.columns = ['source','em1','em2']
# eu_df
Email_temp_df = pd.merge(Email_df,Email_emb_df)
Email_temp_df['degree'] =Email_temp_df['source'].map(Email_node_degree)

fb_temp_df = pd.merge(fb_df,fb_emb_df)
fb_temp_df['degree'] =fb_temp_df['source'].map(fb_node_degree)
# temp_df
silhoute_avg =[]
n_cluster_range = range(40,50)
for NUM_CLUSTERS in n_cluster_range:
# NUM_CLUSTERS = 10

  kmeans = KMeans(n_clusters=NUM_CLUSTERS)
  kmeans.fit(temp_df)
  cluster_labels = kmeans.labels_
  # print(cluster_labels)
  # print(np.unique(cluster_labels))
  silhoute_avg.append(silhouette_score(temp_df,cluster_labels))
 
plt.figure(figsize=(10,5))
plt.plot(n_cluster_range,silhoute_avg,'bx-')
plt.xlabel('Values of K') 
plt.ylabel('Silhouette score') 
plt.title('Silhouette analysis For Optimal k')
plt.show()

Email_kmeans = KMeans(n_clusters=42)
Email_kmeans.fit(Email_temp_df)
labels = Email_kmeans.predict(Email_temp_df)
Email_temp_df['cluster'] = labels
Email_temp_df

fb_kmeans = KMeans(n_clusters=80)
fb_kmeans.fit(fb_temp_df)
labels = fb_kmeans.predict(fb_temp_df)
fb_temp_df['cluster'] = labels
fb_temp_df

Email_temp1 = Email_temp_df.groupby(by='cluster').agg('count').sort_values('source', ascending=False)
fb_temp1 = fb_temp_df.groupby(by='cluster').agg('count').sort_values('source', ascending=False)
Email_sig_clusters = list(Email_temp1[:30].index)
fb_sig_clusters = list(fb_temp1[:60].index)
 
Eamil_temp2 = Email_temp_df[Email_temp_df.cluster.isin(Email_sig_clusters)]
fb_temp2 = fb_temp_df[fb_temp_df.cluster.isin(fb_sig_clusters)]
fb_temp2.head(20)

# Email EU Core Graph Greedy Celf++
graph=Email_graph
df = Eamil_temp2 
# greedy_spreads1=[] 
# greedy_elapsed1=[]  
# celf_elapsed1=[]
# celf_spreads1=[] 
# celfpp_spreads1=[] 
# celfpp_elapsed1=[]
simulated_annealing_spreads1=[] 
simulated_annealing_elapsed1=[]
simulated_annealing_solution1=[] 
genetic_algorithm_spreads1=[] 
genetic_algorithm_elapsed1=[]
genetic_algorithm_solution1=[]
opt_genetic_algorithm_spreads1=[] 
opt_genetic_algorithm_elapsed1=[]
opt_genetic_algorithm_solution1=[]

lower_seed_size,higher_seed_size=50,200
for i in tqdm(range(lower_seed_size,higher_seed_size,10)):
    k = i
    prob = 0.5
    n_iters =200
    generation_count=80
    population_size=800
   
    # greedy_solution, greedy_spreads, greedy_elapsed = greedy_algorithm.algo(graph, k, prob, n_iters)
    # celf_solution, celf_spreads, celf_elapsed = celf.algo(graph, k, prob, n_iters)
    
    # celfpp_solution, celfpp_spreads, celfpp_elapsed = celfpp.algo(graph, k, prob, n_iters)
    
    opt_genetic_algorithm_solution,opt_genetic_algorithm_spreads,opt_genetic_algorithm_elapsed=genetic_im_opt.algo(graph,df,generation_count,population_size,k)
    genetic_algorithm_solution,genetic_algorithm_spreads,genetic_algorithm_elapsed=genetic_algorithm.algo(graph,generation_count,population_size,k)
    simulated_annealing_solution,simulated_annealing_spreads,simulated_annealing_elapsed=simulated_annealing.algo(graph,k,prob,n_iters)

    
    
 
    
    # greedy_spreads1.append(greedy_spreads)
    # greedy_elapsed1.append(greedy_elapsed)
    
    # celf_spreads1.append(celf_spreads) 
    # celf_elapsed1.append(celf_elapsed)
    # celfpp_spreads1.append(celfpp_spreads) 
    # celfpp_elapsed1.append(celfpp_elapsed)
    
    simulated_annealing_spreads1.append(simulated_annealing_spreads) 
    simulated_annealing_elapsed1.append(simulated_annealing_elapsed)
    simulated_annealing_solution1.append(simulated_annealing_solution)
    genetic_algorithm_spreads1.append(genetic_algorithm_spreads) 
    genetic_algorithm_elapsed1.append(genetic_algorithm_elapsed)
    genetic_algorithm_solution1.append(genetic_algorithm_solution)
    opt_genetic_algorithm_spreads1.append(opt_genetic_algorithm_spreads) 
    opt_genetic_algorithm_elapsed1.append(opt_genetic_algorithm_elapsed)
    opt_genetic_algorithm_solution1.append(opt_genetic_algorithm_solution)
   